### Transforming SHIPPED & RECEIVED File

In [1]:
import pandas as pd 
import numpy as np
import re
import os

#### Uploading and Parsing columns needed

In [2]:
#df = pd.read_csv('C:\ProjectFiles\shipped received.csv', encoding = 'unicode_escape')
df = pd.read_csv('C:\ProjectFiles\STB SHIPPED & RECEIVED3.csv', encoding = 'unicode_escape')

C:\Users\Jose\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.columns
df[['DATE', 'SITE', 'WORK ORDER', 'EQUIPMENT', 'SN#',
       'CONDITION', 'TRK#', 'PRP TRK#','DATE.1',
       'SITE.1', 'WORK ORDER.1', 'EQUIPMENT.1', 'SN#.1', 'CONDITION.1',
       'PRP TRK#.1']].columns
df.rename(columns={'WORK ORDER':'SHIP SVO','EQUIPMENT':'EQPMT SHIPPED','TRK#':'TRK','PRP TRK#':'PRP TRK','DATE.1':'RCVD','SITE.1':'SITE RCVD','WORK ORDER.1':'RCVD SVO','EQUIPMENT.1':'EQPMT RCVD','CONDITION.1':'CONDITION RCVD','PRP TRK#.1':'RCVD TRK'},inplace=True)
df=df[['DATE', 'SITE', 'SHIP SVO', 'EQPMT SHIPPED',
       'CONDITION', 'TRK', 'PRP TRK','RCVD','SITE RCVD', 'RCVD SVO', 'EQPMT RCVD','CONDITION RCVD', 'RCVD TRK']]
df

,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK
0,08/22/19,S8469,SVO72298,7 DMM,#NAME?,113947276315,9612018119359200255503,NaN,NaN,NaN,NaN,NaN,NaN
1,08/22/19,S3878,SVO71114,7 DMM,USED,113947276430,9612018119359200255510,09/10/19,S3878,SVO71114,1 3000,USED,1193592002555101193592
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/10/19,S3878,SVO71114,6 5500,USED,1193592002555101193592
3,08/22/19,S7107,SVO72360,2 3501,USED,113947276440,9612018119359200255527,05/17/22,S7107,SVO150952,10 3501,USED,9612018119359200284626
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/22/19,S7731,SVO67348,2 FRANKS,USED,9612018119359200255015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
exchange = df.copy()

In [5]:
exchanges_original = exchange.copy()

In [6]:
## REMOVING EMPTY ROWS AND CHANGING DATE COLUMN TO DATE DATA TYPE

print(exchanges_original.shape)
exchanges_original.drop([3965],axis=0,inplace=True)
empty_idx = exchanges_original[(exchanges_original['EQPMT SHIPPED'].isna()) & (exchanges_original['EQPMT RCVD'].isna())].index
no_empty = exchanges_original.index.drop(empty_idx)
exchanges = exchanges_original.loc[no_empty]
print(exchanges.shape)
exchanges['DATE'] = pd.to_datetime(exchanges['DATE'])
exchanges.info()

(1048575, 13)
(4665, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4665 entries, 0 to 4799
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            4078 non-null   datetime64[ns]
 1   SITE            4077 non-null   object        
 2   SHIP SVO        4088 non-null   object        
 3   EQPMT SHIPPED   4221 non-null   object        
 4   CONDITION       4191 non-null   object        
 5   TRK             4044 non-null   object        
 6   PRP TRK         4042 non-null   object        
 7   RCVD            3402 non-null   object        
 8   SITE RCVD       3731 non-null   object        
 9   RCVD SVO        2951 non-null   object        
 10  EQPMT RCVD      3565 non-null   object        
 11  CONDITION RCVD  2995 non-null   object        
 12  RCVD TRK        3026 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 510.2+ KB


#### Wrangling the Data

In [7]:
# Removing where mentions of a Purchase on RCVD side

purchase_rcvd = exchanges[(exchanges['SITE RCVD'].str.lower().str.contains('.*urchas.*')==True) |
(exchanges['RCVD SVO'].str.lower().str.contains('.*urchas.*')==True) | 
(exchanges['EQPMT RCVD'].str.lower().str.contains('.*urchas.*')==True) |
(exchanges['SITE RCVD'].str.lower().str.contains('.*sold.*')==True) |
(exchanges['RCVD SVO'].str.lower().str.contains('.*sold.*')==True) | 
(exchanges['EQPMT RCVD'].str.lower().str.contains('.*sold.*')==True)]
                              

purchase_rcvd_index=purchase_rcvd.index

result1=exchanges.index.drop(purchase_rcvd_index)
result1
exchanges1=exchanges.loc[result1] 
exchanges1





# Removing where EQPMT Shipped and EQPMT Rcvd are both empty

exchanges32= exchanges1[(exchanges1['EQPMT SHIPPED'].isna()) & (exchanges1['EQPMT RCVD'].isna())]

EQPMT_SHIPPED_null_index = exchanges32.index
drop_null32= exchanges1.index.drop(EQPMT_SHIPPED_null_index)
exchanges23= exchanges1.loc[drop_null32]
print(exchanges23.shape)

exchanges22= exchanges23[(exchanges23['CONDITION'].isna()) & (exchanges23['EQPMT RCVD'].isna())]

condition_null_index = exchanges22.index
drop_null22= exchanges23.index.drop(condition_null_index)
exchanges2= exchanges23.loc[drop_null22]


# 'NEW' in CONDITION, SHIP SVO is empty, CONDITION does not include 'Exchange'
new_condtion_svo_null=exchanges2[(exchanges2['CONDITION'].str.lower().str.contains('.*new.*')==True) & (exchanges2['CONDITION'].str.lower().str.contains('.*exch.*')!=True)&(exchanges2['SHIP SVO'].isna())]
new_condtion_svo_null_index=new_condtion_svo_null.index
drop_index12=exchanges2.index.drop(new_condtion_svo_null_index)
exchanges2A=exchanges2.loc[drop_index12]


# Replacing columns for rows that are empty and do not want removed with 'PLACHOLDER'
rcvdsvo_na = exchanges2A[(exchanges2A['SITE RCVD'].notna()) & (exchanges2A['EQPMT RCVD'].notna())&(exchanges2A['RCVD SVO'].isna())].index
exchanges2A.loc[rcvdsvo_na,'RCVD SVO'] = exchanges2A.loc[rcvdsvo_na,'RCVD SVO'].fillna('PLACEHOLDER')

rcvdtrk_na = exchanges2A[(exchanges2A['SITE RCVD'].notna()) & (exchanges2A['EQPMT RCVD'].notna())&(exchanges2A['RCVD TRK'].isna())].index
exchanges2A.loc[rcvdtrk_na,'RCVD TRK'] = exchanges2A.loc[rcvdtrk_na,'RCVD TRK'].fillna('PLACEHOLDER')



# Removing rows that have either Shipped OR Received side empty but have the rest filled in. Will be used at the end to concatenate
exchangesA02 = exchanges2A[(exchanges2A['SITE'].isna()) & (exchanges2A['SHIP SVO'].isna()) & (exchanges2A['EQPMT SHIPPED'].isna()) &(exchanges2A['RCVD TRK'].notna())].index
drop_nullA02 = exchanges2A.index.drop(exchangesA02)
exchangesA2 = exchanges2A.loc[drop_nullA02]
print(exchangesA2.shape)


exchangesA003 = exchangesA2[(exchangesA2['SITE RCVD'].isna()) & (exchangesA2['RCVD SVO'].isna()) & (exchangesA2['SHIP SVO'].notna()) & (exchangesA2['EQPMT RCVD'].isna())& (exchangesA2['RCVD TRK'].isna()) &(exchangesA2['PRP TRK'].notna()) &(exchangesA2['PRP TRK'].str.lower().str.contains('.*ol.*')!=True)].index
drop_nullA003 = exchangesA2.index.drop(exchangesA003)
exchangesA03 = exchangesA2.loc[drop_nullA003]
print(exchangesA003.shape)

exchangesA0003 = exchangesA03[(exchangesA03['SITE RCVD'].isna()) & (exchangesA03['RCVD SVO'].isna()) & (exchangesA03['EQPMT RCVD'].isna()) &(exchangesA03['PRP TRK'].notna())&(exchangesA03['PRP TRK'].str.lower().str.contains('.*ol.*')==True)].index
drop_nullA0003 = exchangesA03.index.drop(exchangesA0003)
exchangesA3 = exchangesA2.loc[drop_nullA0003]
print(exchangesA0003.shape)


# Forward filling for rows that have EQPMT Shipped and same SVO across multiple rows althought empty on CSV file
cols = ['SITE','SHIP SVO','CONDITION RCVD']

exchangesA3.loc[:,cols] = exchangesA3.loc[:,cols].fillna(method='ffill',limit=1)

exchangesA3.loc[:,cols] = exchangesA3.loc[:,cols].fillna(method='ffill',limit=1)







# ALL PURCHASES ON SHIPPED SIDE AND NULLS ON SITE_RCVD
ship_purchase_and_null_siteA3 = pd.concat([exchangesA3[(exchangesA3['CONDITION'].str.lower().str.contains('.*urchas.*')==True) & (exchangesA3['SITE RCVD'].isna())&(exchangesA3['RCVD TRK'].isna())&(exchangesA3['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)], 
exchangesA3[(exchangesA3['PRP TRK'].str.lower().str.contains('.*urchas.*')==True) & (exchangesA3['EQPMT RCVD'].isna())&(exchangesA3['RCVD TRK'].isna())&(exchangesA3['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],
exchangesA3[(exchangesA3['CONDITION'].str.lower().str.contains('.*ol.*')==True) & (exchangesA3['EQPMT RCVD'].isna())&(exchangesA3['RCVD TRK'].isna())&(exchangesA3['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],
exchangesA3[(exchangesA3['PRP TRK'].str.lower().str.contains('.*ol.*')==True) & (exchangesA3['EQPMT RCVD'].isna())&(exchangesA3['RCVD TRK'].isna()) &(exchangesA3['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)]])

ship_purchase_and_null_siteA3_index = ship_purchase_and_null_siteA3.index.drop_duplicates()
drop_index0A3 = exchangesA3.index.drop(ship_purchase_and_null_siteA3_index)
drop_index0A3
exchanges2B = exchangesA3.loc[drop_index0A3]
print(exchangesA3.shape)
print(exchanges2B.shape)



ship_purchase_and_null_svo = pd.concat([exchanges2B[(exchanges2B['CONDITION'].str.lower().str.contains('.*urchas.*')==True) &(exchanges2B['RCVD TRK'].isna()) &(exchanges2B['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)], 
exchanges2B[(exchanges2B['PRP TRK'].str.lower().str.contains('.*urchas.*')==True) &(exchanges2B['RCVD TRK'].isna()) &(exchanges2B['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],  #MAYBE ADD SHIP SVO ISNA()
exchanges2B[(exchanges2B['CONDITION'].str.lower().str.contains('.*ol.*')==True) &(exchanges2B['RCVD TRK'].isna()) &(exchanges2B['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],
exchanges2B[(exchanges2B['PRP TRK'].str.lower().str.contains('.*ol.*')==True) &(exchanges2B['RCVD TRK'].isna()) &(exchanges2B['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)]])

ship_purchase_and_null_svo_index = ship_purchase_and_null_svo.index.drop_duplicates()
drop_index2B = exchanges2B.index.drop(ship_purchase_and_null_svo_index)
exchanges2C = exchanges2B.loc[drop_index2B]
print(exchanges2C.shape)


exchanges3=exchanges2C.copy()










# Forward filling remaining columns after filling in SVOs where neded & removing duplicate columns after merging

exchanges3['DATE'] = pd.to_datetime(exchanges3['DATE'])
exchanges3['RCVD'] = pd.to_datetime(exchanges3['RCVD'])



exchanges074 = exchanges3.fillna('').groupby(['SHIP SVO'],sort=False)['RCVD TRK'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})
exchanges064 = exchanges3.fillna('').groupby(['SHIP SVO'],sort=False)['EQPMT RCVD'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})
exchanges054 = exchanges3.fillna('').groupby(['SHIP SVO'],sort=False)['RCVD SVO'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})
exchanges044 = exchanges3.fillna('').groupby(['SHIP SVO'],sort=False)['SITE RCVD'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})exchanges034 = exchanges3.fillna('').applymap(str).groupby(['SHIP SVO'],sort=False)['RCVD'].apply(''.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})
exchanges024 = exchanges3.fillna('').groupby(['SHIP SVO'],sort=False)['PRP TRK'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})
exchanges014 = exchanges3.fillna('').groupby(['SHIP SVO'],sort=False)['TRK'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})






exchanges74 = pd.merge(exchanges3,exchanges074,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges74['RCVD TRK'].fillna(exchanges74['RCVD TRK_x'])
exchanges64 = pd.merge(exchanges74,exchanges064,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges64['EQPMT RCVD'].fillna(exchanges64['EQPMT RCVD_x'])
exchanges54 = pd.merge(exchanges64,exchanges054,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges54['RCVD SVO'].fillna(exchanges54['RCVD SVO_x'])
exchanges44 = pd.merge(exchanges54,exchanges044,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges44['SITE RCVD'].fillna(exchanges44['SITE RCVD_x'])
exchanges24 = pd.merge(exchanges44,exchanges024,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges24['PRP TRK'].fillna(exchanges24['PRP TRK_x'])
exchanges4 = pd.merge(exchanges24,exchanges014,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges4['TRK'].fillna(exchanges4['TRK_x'])


exchanges4.drop(exchanges4.filter(regex='_x$').columns.tolist(),axis=1,inplace=True)
exchanges4=exchanges4[['DATE','SITE','SHIP SVO','EQPMT SHIPPED','CONDITION','TRK','PRP TRK','RCVD','SITE RCVD','RCVD SVO','EQPMT RCVD','CONDITION RCVD','RCVD TRK']]
print(exchanges4.shape)
exchanges6=exchanges4.copy()
ship_purchase_and_null_siteA3.drop_duplicates()

(4491, 13)
(4017, 13)
(417,)
(368,)


<ipython-input-7-8583e77cec21>:86: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ship_purchase_and_null_siteA3 = pd.concat([exchangesA3[(exchangesA3['CONDITION'].str.lower().str.contains('.*urchas.*')==True) & (exchangesA3['SITE RCVD'].isna())&(exchangesA3['RCVD TRK'].isna())&(exchangesA3['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],
<ipython-input-7-8583e77cec21>:87: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exchangesA3[(exchangesA3['PRP TRK'].str.lower().str.contains('.*urchas.*')==True) & (exchangesA3['EQPMT RCVD'].isna())&(exchangesA3['RCVD TRK'].isna())&(exchangesA3['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],
<ipython-input-7-8583e77cec21>:88: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exchangesA3[(exchangesA3['CONDITION'].str.lower().str.contains('.*ol.*')==True) & (exchangesA3['EQPMT

(3232, 13)
(3232, 13)


<ipython-input-7-8583e77cec21>:100: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ship_purchase_and_null_svo = pd.concat([exchanges2B[(exchanges2B['CONDITION'].str.lower().str.contains('.*urchas.*')==True) &(exchanges2B['RCVD TRK'].isna()) &(exchanges2B['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],
<ipython-input-7-8583e77cec21>:101: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exchanges2B[(exchanges2B['PRP TRK'].str.lower().str.contains('.*urchas.*')==True) &(exchanges2B['RCVD TRK'].isna()) &(exchanges2B['PRP TRK'].notna())&(exchanges['SITE RCVD'].str.lower().str.contains('.*no exch.*')!=True)],  #MAYBE ADD SHIP SVO ISNA()
<ipython-input-7-8583e77cec21>:102: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exchanges2B[(exchanges2B['CONDITION'].str.lower().str.contains('.*ol.*')==True) &(exchanges2B['RCVD TRK'].isna()) &(exchanges2B['PRP TRK'].not

(3231, 13)
(3231, 13)


,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK


#### Wrangling DATE Columns and Merging EQPMT Shipped as well delicately to avoid merging if Exchange included both and purchase

In [8]:

date_placeholder =  exchanges6[(exchanges6['DATE'].isna()) & (exchanges6['RCVD'].isna())].fillna('2023-01-01')
exchanges07=pd.concat([exchanges6,date_placeholder])
exchanges07.reset_index(inplace=True)

drop_dates_na_index= exchanges07[(exchanges07['DATE'].isna()) & (exchanges07['RCVD'].isna())].index
drop_dates_na= exchanges07.index.drop(drop_dates_na_index)
exchanges7 = exchanges07.loc[drop_dates_na]
## CONCATENATING MULTIPLE ROWS THAT CONSIST OF EXCHANGES (5)

exchanges6[(exchanges6['SHIP SVO']=='SVO148366')]

print(exchanges07.shape)
print(exchanges7.shape)



exchanges7[(exchanges7['SHIP SVO']=='SVO148366')]
#exchanges7[exchanges7['SHIP SVO']=='SVO164555'], this is a purchase ticket


exchanges7['DATE'] = pd.to_datetime(exchanges7['DATE'])
exchanges7['RCVD'] = pd.to_datetime(exchanges7['RCVD'])

exchanges08= exchanges7[(exchanges7['DATE']>='01/01/21')|(exchanges7['RCVD']>='01/01/21')]
print(exchanges08.shape)

exchanges08.loc[:,'DATE'] = exchanges08['DATE'].dt.strftime('%Y-%m-%d')
exchanges08.loc[:,'RCVD'] = exchanges08['RCVD'].dt.strftime('%Y-%m-%d')
exchanges8=exchanges08.replace('2023-01-01',np.nan)


purchases = ['purchase','sold']

duplicates = exchanges8[(exchanges8['CONDITION'].str.lower().str.contains('|'.join(purchases))==True)]['SHIP SVO'] #& (exchanges8['SHIP SVO'].duplicated())]['SHIP SVO'] #date is null
duplicates2=[]
duplicates2.extend(duplicates.copy())
duplicates2
pd.Series(duplicates2)
exchanges009=exchanges8.loc[~exchanges8['SHIP SVO'].isin(pd.Series(duplicates2))]
exchanges09=exchanges009.fillna('').groupby(['SHIP SVO'],sort=False)['EQPMT SHIPPED'].apply('/'.join).reset_index()
exchanges9 = pd.merge(exchanges8,exchanges09,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges9['EQPMT SHIPPED'].fillna(exchanges9['EQPMT SHIPPED_x'],inplace=True)
#search_svos = ['SVO169683','SVO123693']

#exchanges9[(exchanges9['SHIP SVO'].str.contains('|'.join(search_svos))==True)]



exchanges9.drop(exchanges9.filter(regex='_x$').columns.tolist(),axis=1,inplace=True)
exchanges9=exchanges9[['DATE','SITE','SHIP SVO','EQPMT SHIPPED','CONDITION','TRK','PRP TRK','RCVD','SITE RCVD','RCVD SVO','EQPMT RCVD','CONDITION RCVD','RCVD TRK']]


exchanges10A = exchanges9.loc[exchanges9['SHIP SVO'].isin(pd.Series(duplicates2))]
exchanges10A[(exchanges10A).duplicated()]
exchanges10A_drop_duplicates_index=exchanges10A[(exchanges10A).duplicated()].index
exchanges10A_dropped_duplicates= exchanges9.index.drop(exchanges10A_drop_duplicates_index)
exchanges10=exchanges9.loc[exchanges10A_dropped_duplicates]
print(exchanges10.shape)




exchanges11A = exchanges10.fillna('').groupby(['SHIP SVO'],sort=False)['DATE'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})
exchanges11B = exchanges10.fillna('').groupby(['SHIP SVO'],sort=False)['RCVD'].apply('/'.join).reset_index() #agg({['EQPMT SHIPPED'].values:''.join})

exchanges11A2 = pd.merge(exchanges10,exchanges11A,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges11A2['DATE'].fillna(exchanges11A2['DATE_x'],inplace=True)
exchanges11B2 = pd.merge(exchanges11A2,exchanges11B,on='SHIP SVO', how='outer',suffixes=('_x',''))
exchanges11B2['RCVD'].fillna(exchanges11B2['RCVD_x'],inplace=True)

exchanges11B2.drop(exchanges11B2.filter(regex='_x$').columns.tolist(),axis=1,inplace=True)
exchanges11=exchanges11B2[['DATE','SITE','SHIP SVO','EQPMT SHIPPED','CONDITION','TRK','PRP TRK','RCVD','SITE RCVD','RCVD SVO','EQPMT RCVD','CONDITION RCVD','RCVD TRK']]
print(exchanges11.shape)




exchanges12A= exchanges11[(exchanges11).duplicated()].index
exchanges12A_dropped = exchanges11.index.drop(exchanges12A)
exchanges12 = exchanges11.loc[exchanges12A_dropped]
print(exchanges12.shape)

exchanges13A = exchanges12[(exchanges12['DATE']=='')].index
exchanges13A_dropped = exchanges12.index.drop(exchanges13A)
exchanges13= exchanges12.loc[exchanges13A_dropped]
print(exchanges13.shape)
# exchanges11.loc[exchanges12A]


(3322, 14)
(3231, 14)
(2202, 14)


C:\Users\Jose\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


(2202, 13)
(2202, 13)
(2129, 13)
(2114, 13)


In [9]:
## Verifying there are no more Purchases on both SHIPPED & RECEIVED portions

pd.concat([exchanges13[(exchanges13['CONDITION'].str.lower().str.contains('.*urchas.*')==True) & (exchanges13['SITE RCVD'].str.lower().str.contains('.*urchas.*')!=True) & (exchanges13['RCVD SVO'].str.lower().str.contains('.*urchas.*')!=True) & (exchanges13['EQPMT RCVD'].str.lower().str.contains('.*urchas.*')!=True) & exchanges13['SHIP SVO'].isna()], 
exchanges13[(exchanges13['PRP TRK'].str.lower().str.contains('.*urchas.*')==True) & (exchanges13['SITE RCVD'].str.lower().str.contains('.*urchas.*')!=True)& (exchanges13['RCVD SVO'].str.lower().str.contains('.*urchas.*')!=True) & (exchanges13['EQPMT RCVD'].str.lower().str.contains('.*urchas.*')!=True) & exchanges13['SHIP SVO'].isna()],
exchanges13[(exchanges13['CONDITION'].str.lower().str.contains('.*ol.*')==True)  & (exchanges13['SITE RCVD'].str.lower().str.contains('.*urchas.*')!=True) & (exchanges13['RCVD SVO'].str.lower().str.contains('.*urchas.*')!=True) & (exchanges13['EQPMT RCVD'].str.lower().str.contains('.*urchas.*')!=True) & exchanges13['SHIP SVO'].isna()], 
exchanges13[(exchanges13['PRP TRK'].str.lower().str.contains('.*ol.*')==True) & (exchanges13['SITE RCVD'].str.lower().str.contains('.*urchas.*')!=True) & (exchanges13['RCVD SVO'].str.lower().str.contains('.*urchas.*')!=True) & (exchanges13['EQPMT RCVD'].str.lower().str.contains('.*urchas.*')!=True) & exchanges13['SHIP SVO'].isna()]]).drop_duplicates()

,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK


#### Concatenating rows we removed earlier that had 1 side filled but not the other & NOT a purchase order

In [10]:
exchanges6A = exchanges2A.loc[exchangesA02]
print(exchanges6A.shape)
exchanges6A
exchanges6B = exchangesA2.loc[exchangesA003]
print(exchanges6A.shape)
print(exchanges6B.shape)


## Separate all STB Shipments from the past 60 days. This may be added last as long as we REMOVE dupllicates
exchanges6A['RCVD'] = pd.to_datetime(exchanges6A['RCVD'])
exchanges6A['DATE'] = pd.to_datetime(exchanges6A['DATE'])
from datetime import datetime,timedelta
today=datetime.now()
days180=exchanges6A[exchanges6A.RCVD > today - pd.to_timedelta('537day')]
print(days180.shape)
days180['RCVD']=days180['RCVD'].dt.date

print(exchanges13.shape)
exchanges14 = pd.concat([exchanges13,days180]).reset_index(drop=True)
print(exchanges14.shape)


duplicates2A = exchanges14[(exchanges14['DATE'].notna())&(exchanges14['SHIP SVO'].duplicated())]['SHIP SVO']
duplicates3=[]
duplicates3.extend(duplicates2A.copy())
len(duplicates3)
exchanges14A = exchanges14.loc[exchanges14['SHIP SVO'].isin(pd.Series(duplicates2A))]
exchanges14A
exchanges14A_drop_duplicates_index=exchanges14A[(exchanges14A).duplicated()].index
exchanges14A_dropped_duplicates= exchanges14.index.drop(exchanges14A_drop_duplicates_index)
exchanges15=exchanges14.loc[exchanges14A_dropped_duplicates]
print(exchanges15.shape)

exchanges6B['RCVD'] = pd.to_datetime(exchanges6B['RCVD'])
exchanges6B['DATE'] = pd.to_datetime(exchanges6B['DATE'])
from datetime import datetime,timedelta
today=datetime.now()
days180B=exchanges6B[exchanges6B.DATE > today - pd.to_timedelta('537day')]
print(days180B.shape)
days180B['DATE']=days180B['DATE'].dt.date

exchanges16 = pd.concat([exchanges15,days180B]).reset_index(drop=True)
print(exchanges16.shape)
duplicates2B = exchanges16[(exchanges16['DATE'].notna())&(exchanges16['SHIP SVO'].duplicated())]['SHIP SVO']
duplicates3B=[]
duplicates3B.extend(duplicates2B.copy())
len(duplicates3B)
exchanges16A = exchanges16.loc[exchanges16['SHIP SVO'].isin(pd.Series(duplicates2B))]
print(exchanges16A.shape)
exchanges16A_drop_duplicates_index=exchanges16A[(exchanges16A).duplicated()].index
exchanges16A_dropped_duplicates= exchanges16.index.drop(exchanges16A_drop_duplicates_index)
exchanges17=exchanges16.loc[exchanges16A_dropped_duplicates]
print(exchanges17.shape)
exchanges16[exchanges16.duplicated()]

(438, 13)
(438, 13)
(417, 13)
(124, 13)
(2114, 13)
(2238, 13)
(2238, 13)
(177, 13)
(2415, 13)
(157, 13)
(2415, 13)


<ipython-input-10-8f4e0c260d60>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days180['RCVD']=days180['RCVD'].dt.date
<ipython-input-10-8f4e0c260d60>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days180B['DATE']=days180B['DATE'].dt.date


,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK


##### Removing any remaining purchases

In [11]:
last_removal = exchanges17[(exchanges17['CONDITION'].str.lower().str.contains('.*urchas.*')==True) | 
(exchanges17['PRP TRK'].str.lower().str.contains('.*urchas.*')==True) |
(exchanges17['CONDITION'].str.lower().str.contains('.*ol.*')==True) |
(exchanges17['PRP TRK'].str.lower().str.contains('.*ol.*')==True)] 

las_removal_index = last_removal.index
drop_index17 = exchanges17.index.drop(las_removal_index)
exchanges18 = exchanges17.loc[drop_index17]
print(exchanges18.shape)
exchanges18

(2283, 13)


,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK
0,2019-08-22,S7107,SVO72360,2 3501,USED,113947276440,9612018119359200255527,2022-05-17,S7107,SVO150952,10 3501,USED,9612018119359200284626
1,2019-08-26,S4823,SVO71620,22 DMM,USED,113947277300,N/A (INTERNATIONAL),2021-10-13,S4823 This completes Exchgs on SVO138741,& SVO156050 leaving 15 towards this Exchg,35 DMM,USED,1010069711240007704200284664587538
2,2019-09-20,S8351,SVO74805,1 DMM,USED,113947282343,9612018119359200256494,2022-02-07,S8351,SVO161188,3 DMM,USED,9612018119359200301231
3,2019-11-08,S8214,SVO79292,2 3501,USED,113947293903,N/A INTERNATIONAL,2022-02-22,S8214,PLACEHOLDER,1 DMM / 11 3501,USED,1043241552740007704200518619232608
4,2019-11-11,S7753,SVO79606,5 DMM,USED,113947294358,9612018119359200258498,2021-08-08,S7753,PLACEHOLDER,5 FRANKS 4 Damaged,USED,9612018119359200270490
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2410,2022-06-07,C00445,SVO186470,2 5500,USED,518619255296,9612018119359200315566,NaN,NaN,NaN,NaN,NaN,NaN
2411,2022-06-07,C00498,SVO186499,5 5500,USED,518619255311,9612018119359200315573,NaN,NaN,NaN,NaN,NaN,NaN
2412,2022-06-07,S7758,SVO186564,5 3501,USED,518619255425,9612018119359200315580,NaN,NaN,NaN,NaN,NaN,NaN
2413,2022-06-07,C00239,SVO186581,2 DMM,USED,518619255300,9612018119359200315740,NaN,NaN,NaN,NaN,NaN,NaN


###### Stripping any unnecessary '/'

In [12]:
cols= ['DATE', 'SITE', 'SHIP SVO', 'EQPMT SHIPPED', 'CONDITION', 'TRK','PRP TRK', 'RCVD', 'SITE RCVD', 'RCVD SVO', 'EQPMT RCVD','CONDITION RCVD', 'RCVD TRK']

exchanges18['DATE']= exchanges18['DATE'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['SITE']= exchanges18['SITE'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['SHIP SVO']= exchanges18['SHIP SVO'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['EQPMT SHIPPED']= exchanges18['EQPMT SHIPPED'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['CONDITION']= exchanges18['CONDITION'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['TRK']= exchanges18['TRK'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['PRP TRK']= exchanges18['PRP TRK'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['RCVD']= exchanges18['RCVD'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['SITE RCVD']= exchanges18['SITE RCVD'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['RCVD SVO']= exchanges18['RCVD SVO'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['EQPMT RCVD']= exchanges18['EQPMT RCVD'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['CONDITION RCVD']= exchanges18['CONDITION RCVD'].apply(str).replace(r'/$','',regex=True).str.strip()
exchanges18['RCVD TRK']= exchanges18['RCVD TRK'].apply(str).replace(r'/$','',regex=True).str.strip()

exchanges18['DATE']= exchanges18['DATE'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['SITE']= exchanges18['SITE'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['SHIP SVO']= exchanges18['SHIP SVO'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['EQPMT SHIPPED']= exchanges18['EQPMT SHIPPED'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['CONDITION']= exchanges18['CONDITION'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['TRK']= exchanges18['TRK'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['PRP TRK']= exchanges18['PRP TRK'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['RCVD']= exchanges18['RCVD'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['SITE RCVD']= exchanges18['SITE RCVD'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['RCVD SVO']= exchanges18['RCVD SVO'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['EQPMT RCVD']= exchanges18['EQPMT RCVD'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['CONDITION RCVD']= exchanges18['CONDITION RCVD'].apply(str).replace(r'^/','',regex=True).str.strip()
exchanges18['RCVD TRK']= exchanges18['RCVD TRK'].apply(str).replace(r'^/','',regex=True).str.strip()

In [13]:
# Testing a few SVOs
search_svo = ['SVO169683','SVO169686','SVO170341','SVO136238','SVO124332','SVO146849','SVO123693','SVO132335','SVO149359','SVO157883','SVO182413','SVO125609','SVO183062']
search_svos = ['SVO132335','SVO157883']
search_svos2 = ['This returns','Completes','These returns']
search_svos3 = ['SVO169995','SVO167285','SVO157883']
search_svos4 = ['SVO132703']

exchanges18[(exchanges18['SHIP SVO'].str.contains('|'.join(search_svos4))==True)]

,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK
2244,2021-04-20,S8391,SVO132703,5 DMM,NEW,190439955389,9612018119359200268626,nan,nan,nan,nan,nan,nan


In [14]:
# Checking for any duplicates
exchanges18[exchanges18.duplicated()]

,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK


In [15]:
exchanges18

,DATE,SITE,SHIP SVO,EQPMT SHIPPED,CONDITION,TRK,PRP TRK,RCVD,SITE RCVD,RCVD SVO,EQPMT RCVD,CONDITION RCVD,RCVD TRK
0,2019-08-22,S7107,SVO72360,2 3501,USED,113947276440,9612018119359200255527,2022-05-17,S7107,SVO150952,10 3501,USED,9612018119359200284626
1,2019-08-26,S4823,SVO71620,22 DMM,USED,113947277300,N/A (INTERNATIONAL),2021-10-13,S4823 This completes Exchgs on SVO138741,& SVO156050 leaving 15 towards this Exchg,35 DMM,USED,1010069711240007704200284664587538
2,2019-09-20,S8351,SVO74805,1 DMM,USED,113947282343,9612018119359200256494,2022-02-07,S8351,SVO161188,3 DMM,USED,9612018119359200301231
3,2019-11-08,S8214,SVO79292,2 3501,USED,113947293903,N/A INTERNATIONAL,2022-02-22,S8214,PLACEHOLDER,1 DMM / 11 3501,USED,1043241552740007704200518619232608
4,2019-11-11,S7753,SVO79606,5 DMM,USED,113947294358,9612018119359200258498,2021-08-08,S7753,PLACEHOLDER,5 FRANKS 4 Damaged,USED,9612018119359200270490
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2410,2022-06-07,C00445,SVO186470,2 5500,USED,518619255296,9612018119359200315566,nan,nan,nan,nan,nan,nan
2411,2022-06-07,C00498,SVO186499,5 5500,USED,518619255311,9612018119359200315573,nan,nan,nan,nan,nan,nan
2412,2022-06-07,S7758,SVO186564,5 3501,USED,518619255425,9612018119359200315580,nan,nan,nan,nan,nan,nan
2413,2022-06-07,C00239,SVO186581,2 DMM,USED,518619255300,9612018119359200315740,nan,nan,nan,nan,nan,nan


#### Exporting Cleaned Data to a CSV file

In [16]:
exchanges18.to_csv('Shipped_Received_clean.csv',sep=',',index=False)